In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: c:\Users\nives\AllFolders\SML\mlfs-book
HopsworksSettings initialized!


__This notebook should be scheduled to run daily__

The GitHub Action is stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

### Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> Get the Sensor URL, Country, City, Street names from Hopsworks </span>

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_BERLIN_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']

latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-17 14:28:18,993 INFO: Initializing external client
2025-11-17 14:28:18,994 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 14:28:20,632 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279175


'{"country": "Germany", "city": "Berlin", "latitude": 52.52, "longitude": 13.4}'

### <span style="color:#ff5f27;">Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_berlin',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_berlin',
    version=1,
)

### Get streets and urls

In [5]:
metadata = air_quality_fg.select(['street', 'url']).read()
metadata = metadata[['street', 'url']].drop_duplicates()  
metadata

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.10s) 


,street,url
0,friedrichshagen,https://api.waqi.info/feed/@10034
1,neukolln-silbersteinstrasse,https://api.waqi.info/feed/@10036
2,karl-liebknecht-strasse,https://api.waqi.info/feed/@6132
3,wedding-amrumer-strasse,https://api.waqi.info/feed/@10030
4,mariendorf--mariendorfer-damm,https://api.waqi.info/feed/@10040
6,buch,https://api.waqi.info/feed/@10033
10,neukolln-nansenstrasse,https://api.waqi.info/feed/@10032
18,mitte--bruckenstrasse,https://api.waqi.info/feed/@10039
19,karl-marx-strasse,https://api.waqi.info/feed/@13851
29,leipziger-strasse,https://api.waqi.info/feed/@13852


## <span style='color:#ff5f27'> Retrieve today's air quality data (PM2.5) from the AQI API</span>


### Get recent PM2.5 values from Hopsworks

In [8]:
import requests
import pandas as pd

aq_today_df = pd.DataFrame()
for i in range(metadata.shape[0]):
    aqicn_url = metadata.iloc[i]['url']
    street = metadata.iloc[i]['street']

    # Air quality: yesterday
    aq_today_i_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)

    # lagged Air Quality for yesterday
    lagged_days = 3
    pm25_recent_i_df = air_quality_fg.select(['pm25', 'date']) \
                    .filter(air_quality_fg.url == aqicn_url) \
                    .filter(air_quality_fg.date > today - datetime.timedelta(days=lagged_days+1)) \
                    .read()
    pm25_recent_i_df = pm25_recent_i_df[['pm25', 'date']].drop_duplicates()  
    pm25_recent_i_df = pm25_recent_i_df.sort_values(by=['date'], ascending=False)
    pm25_lagged_1 = [ pm25_recent_i_df.iloc[0]['pm25'] ]
    pm25_lagged_2 = [ pm25_recent_i_df.iloc[1]['pm25'] ]
    pm25_lagged_3 = [ pm25_recent_i_df.iloc[2]['pm25'] ]

    aq_today_i_df['pm25_lagged_1'] = pm25_lagged_1[-1].astype('float32')
    aq_today_i_df['pm25_lagged_2'] = pm25_lagged_2[-1].astype('float32')
    aq_today_i_df['pm25_lagged_3'] = pm25_lagged_3[-1].astype('float32')

    aq_today_df = pd.concat([aq_today_df, aq_today_i_df], ignore_index=True)

print("Air Quality Data (today):")
aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.18s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.76s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.74s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.75s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.81s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 
Air Quality Data (today):


,pm25,country,city,street,date,url,pm25_lagged_1,pm25_lagged_2,pm25_lagged_3
0,25.0,Germany,Berlin,friedrichshagen,2025-11-17,https://api.waqi.info/feed/@10034,13.0,16.0,54.0
1,46.0,Germany,Berlin,neukolln-silbersteinstrasse,2025-11-17,https://api.waqi.info/feed/@10036,30.0,16.0,52.0
2,42.0,Germany,Berlin,karl-liebknecht-strasse,2025-11-17,https://api.waqi.info/feed/@6132,17.0,15.0,55.0
3,38.0,Germany,Berlin,wedding-amrumer-strasse,2025-11-17,https://api.waqi.info/feed/@10030,17.0,13.0,47.0
4,53.0,Germany,Berlin,mariendorf--mariendorfer-damm,2025-11-17,https://api.waqi.info/feed/@10040,21.0,18.0,53.0
5,25.0,Germany,Berlin,buch,2025-11-17,https://api.waqi.info/feed/@10033,13.0,11.0,38.0
6,42.0,Germany,Berlin,neukolln-nansenstrasse,2025-11-17,https://api.waqi.info/feed/@10032,17.0,14.0,48.0
7,42.0,Germany,Berlin,mitte--bruckenstrasse,2025-11-17,https://api.waqi.info/feed/@10039,17.0,12.0,47.0
8,46.0,Germany,Berlin,karl-marx-strasse,2025-11-17,https://api.waqi.info/feed/@13851,34.0,19.0,52.0
9,46.0,Germany,Berlin,leipziger-strasse,2025-11-17,https://api.waqi.info/feed/@13852,17.0,17.0,51.0


In [9]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   pm25           10 non-null     float32       
 1   country        10 non-null     object        
 2   city           10 non-null     object        
 3   street         10 non-null     object        
 4   date           10 non-null     datetime64[ns]
 5   url            10 non-null     object        
 6   pm25_lagged_1  10 non-null     float32       
 7   pm25_lagged_2  10 non-null     float32       
 8   pm25_lagged_3  10 non-null     float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 688.0+ bytes


## <span style='color:#ff5f27'> Get Weather Forecast data</span>

In [11]:
# Download yesterday weather data
yesterday = today - datetime.timedelta(days=1)
start_date = yesterday
end_date = yesterday

weather_yesterday_df = util.get_historical_weather(city=city, start_date=start_date, end_date=end_date, latitude=latitude, longitude=longitude)
weather_yesterday_df

Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 30.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-16,6.327166,3.8,12.535549,274.099426,Berlin


In [13]:
# Weather hourly forecast starting from today
weather_hourly_df = util.get_hourly_weather_forecast(city=city, latitude=latitude, longitude=longitude)
weather_hourly_df = weather_hourly_df.set_index('date')

# One daily prediction at 12:00
weather_daily_df = weather_hourly_df.between_time('11:59', '12:01')
weather_daily_df = weather_daily_df.reset_index()
weather_daily_df['date'] = pd.to_datetime(weather_daily_df['date']).dt.date
weather_daily_df['date'] = pd.to_datetime(weather_daily_df['date'])
weather_daily_df['city'] = city
weather_daily_df

Coordinates 52.5°N 13.5°E
Elevation 30.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-17,5.60,0.0,11.709688,315.000092,Berlin
1,2025-11-18,5.20,0.0,18.598450,255.425735,Berlin
2,2025-11-19,4.15,0.0,15.905319,185.194336,Berlin
3,2025-11-20,2.90,0.1,17.873556,260.727478,Berlin
4,2025-11-21,1.05,0.0,7.920000,270.000000,Berlin
5,2025-11-22,0.65,0.0,4.693826,147.528824,Berlin
6,2025-11-23,0.70,0.3,7.208994,182.862350,Berlin


## <span style="color:#ff5f27;">Uploading new data to the Feature Store</span>

In [14]:
# Insert new data
air_quality_fg.insert(aq_today_df)
weather_fg.insert(weather_yesterday_df, wait=True)
weather_fg.insert(weather_daily_df, wait=True)
print("Updated feature groups with yesterday's data and forecast.")


2025-11-17 14:39:18,172 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1718740


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_berlin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/air_quality_berlin_1_offline_fg_materialization/executions
2025-11-17 14:39:33,845 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1724775


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_berlin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/weather_berlin_1_offline_fg_materialization/executions
2025-11-17 14:40:17,991 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 14:40:21,186 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 14:41:41,265 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 14:41:41,435 INFO: Waiting for log aggregation to finish.
2025-11-17 14:41:50,333 INFO: Execution finished successfully.
2025-11-17 14:41:50,534 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1724775


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_berlin_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/weather_berlin_1_offline_fg_materialization/executions
2025-11-17 14:42:18,416 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 14:42:21,643 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 14:43:38,799 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 14:43:38,990 INFO: Waiting for log aggregation to finish.
2025-11-17 14:43:47,576 INFO: Execution finished successfully.
Updated feature groups with yesterday's data and forecast.
